In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow.keras.layers as L
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
IMAGE_DIR = '../input/origalist/Images/Images'

In [3]:
df = pd.read_csv('../input/origalist/OrigaList.csv')
df.drop(['Eye', 'ExpCDR', 'Set'], axis=1, inplace=True)
df.head()

,Filename,Glaucoma
0,001.jpg,0
1,002.jpg,0
2,003.jpg,0
3,004.jpg,0
4,005.jpg,0


In [4]:
data_gen = image.ImageDataGenerator(rescale=1./255,
                                   zoom_range=0.3,
                                   rotation_range=20,
                                   validation_split=0.3,
                                   width_shift_range=0.3)

train_generator = data_gen.flow_from_dataframe(df,
                                              IMAGE_DIR,
                                              x_col='Filename',
                                              y_col='Glaucoma',
                                              target_size=(256, 256),
                                              class_mode='raw',
                                              batch_size=16,
                                              shuffle=True,
                                              subset='training')

val_generator = data_gen.flow_from_dataframe(df,
                                            IMAGE_DIR,
                                            x_col='Filename',
                                            y_col='Glaucoma',
                                            target_size=(256, 256),
                                            class_mode='raw',
                                            batch_size=8,
                                            subset='validation')

Found 455 validated image filenames.
Found 195 validated image filenames.


In [5]:
sum(df['Glaucoma'])/len(df['Glaucoma'])

0.25846153846153846

In [7]:
model = keras.Sequential()
model.add(ResNet50(input_shape=(256, 256, 3), include_top=False, weights='imagenet'))
model.add(L.GlobalAveragePooling2D())
model.add(L.Dropout(0.4))
model.add(L.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc', keras.metrics.AUC()])

94773248/94765736 [==============================] - 1s 0us/step


In [8]:
history = model.fit(train_generator, steps_per_epoch=train_generator.n//train_generator.batch_size,
                   validation_data=val_generator, validation_steps=val_generator.n//val_generator.batch_size,
                   epochs=8)

Epoch 1/8
28/28 [==============================] - 280s 10s/step - loss: 1.0687 - acc: 0.6424 - auc: 0.5127 - val_loss: 1734642.1250 - val_acc: 0.1250 - val_auc: 0.5000
Epoch 2/8
28/28 [==============================] - 262s 9s/step - loss: 0.6912 - acc: 0.6765 - auc: 0.5238 - val_loss: 28.6617 - val_acc: 0.8750 - val_auc: 0.5000
Epoch 3/8
28/28 [==============================] - 265s 9s/step - loss: 0.7092 - acc: 0.6697 - auc: 0.5471 - val_loss: 0.6380 - val_acc: 0.8750 - val_auc: 0.5000
Epoch 4/8
28/28 [==============================] - 256s 9s/step - loss: 0.6534 - acc: 0.6811 - auc: 0.5341 - val_loss: 0.7863 - val_acc: 0.1146 - val_auc: 0.4854
Epoch 5/8
28/28 [==============================] - 266s 9s/step - loss: 0.6687 - acc: 0.6765 - auc: 0.4550 - val_loss: 0.3757 - val_acc: 0.8750 - val_auc: 0.5281
Epoch 6/8
28/28 [==============================] - 256s 9s/step - loss: 0.6729 - acc: 0.6378 - auc: 0.4865 - val_loss: 0.6927 - val_acc: 0.5156 - val_auc: 0.4932
Epoch 7/8
28/28 [===